##Unizipping Dataset


In [ ]:
!unzip '/content/drive/MyDrive/archive (4).zip'

Archive:  /content/drive/MyDrive/archive (4).zip
  inflating: test_data/test_data/blasti/DSC_6396.jpg  
  inflating: test_data/test_data/blasti/DSC_6397.jpg  
  inflating: test_data/test_data/blasti/DSC_6398.jpg  
  inflating: test_data/test_data/blasti/DSC_6399.jpg  
  inflating: test_data/test_data/blasti/DSC_6400.jpg  
  inflating: test_data/test_data/blasti/DSC_6401.jpg  
  inflating: test_data/test_data/blasti/DSC_6402.jpg  
  inflating: test_data/test_data/blasti/DSC_6403.jpg  
  inflating: test_data/test_data/blasti/DSC_6405.jpg  
  inflating: test_data/test_data/blasti/DSC_6406.jpg  
  inflating: test_data/test_data/blasti/DSC_6407.jpg  
  inflating: test_data/test_data/blasti/DSC_6408.jpg  
  inflating: test_data/test_data/blasti/DSC_6409.jpg  
  inflating: test_data/test_data/blasti/DSC_6410.jpg  
  inflating: test_data/test_data/blasti/DSC_6411.jpg  
  inflating: test_data/test_data/bonegl/DSC_4587.jpg  
  inflating: test_data/test_data/bonegl/DSC_4588.jpg  
  inflating: tes

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##Data Augmentation

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_gen =  ImageDataGenerator(rescale=1./255,horizontal_flip=True,shear_range = 0.2)
test_gen =  ImageDataGenerator(rescale=1./255)

In [ ]:
train = train_gen.flow_from_directory('/content/train_data/train_data',
                                      target_size=(224,224),
                                      class_mode = 'categorical',
                                      batch_size=8)

test = train_gen.flow_from_directory('/content/test_data/test_data',
                                      target_size=(224,224),
                                      class_mode = 'categorical',
                                      batch_size=8)

Found 150 images belonging to 16 classes.
Found 157 images belonging to 16 classes.


##CNN

In [ ]:
from tensorflow.keras.layers import Convolution2D,MaxPooling2D,Flatten,Dense
from tensorflow.keras.models import Sequential

In [ ]:
model = Sequential()
model.add(Convolution2D(30,(3,3),20,activation = 'relu',input_shape =(224,224,3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(80,activation='relu'))
model.add(Dense(16,activation='softmax'))

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit(train,batch_size=8,validation_data = test,epochs=10)

Epoch 1/10
19/19 [==============================] - 130s 7s/step - loss: 2.7577 - accuracy: 0.0933 - val_loss: 2.6848 - val_accuracy: 0.1465
Epoch 2/10
19/19 [==============================] - 121s 7s/step - loss: 2.6480 - accuracy: 0.1667 - val_loss: 2.6321 - val_accuracy: 0.1401
Epoch 3/10
19/19 [==============================] - 120s 7s/step - loss: 2.5455 - accuracy: 0.2000 - val_loss: 2.6060 - val_accuracy: 0.1656
Epoch 4/10
19/19 [==============================] - 121s 7s/step - loss: 2.4714 - accuracy: 0.2933 - val_loss: 2.5620 - val_accuracy: 0.1592
Epoch 5/10
19/19 [==============================] - 88s 5s/step - loss: 2.3712 - accuracy: 0.2933 - val_loss: 2.6019 - val_accuracy: 0.2102
Epoch 6/10
19/19 [==============================] - 91s 5s/step - loss: 2.3123 - accuracy: 0.3000 - val_loss: 2.5381 - val_accuracy: 0.2357
Epoch 7/10
19/19 [==============================] - 120s 7s/step - loss: 2.2334 - accuracy: 0.2867 - val_loss: 2.5214 - val_accuracy: 0.2102
Epoch 8/10
19/1

## VGG16 Model - For Better Acccuracy

In [ ]:
from tensorflow.keras.layers import Dense,Flatten,Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input

In [ ]:
vgg = VGG16(include_top=False,weights='imagenet',input_shape=(224,224,3))

58889256/58889256 [==============================] - 2s 0us/step


In [ ]:
for layer in vgg.layers:
  layer.trainable = False


In [ ]:
x = Flatten()(vgg.output)

In [ ]:
prediction = Dense(16,activation='softmax')(x)

In [ ]:
model2 = Model(inputs=vgg.input,outputs=prediction)

In [ ]:
model2.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
model2.fit_generator(train,validation_data=test,epochs=10,steps_per_epoch=len(train),validation_steps=len(test))

<ipython-input-20-93be31fd6a36>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model2.fit_generator(train,validation_data=test,epochs=10,steps_per_epoch=len(train),validation_steps=len(test))


Epoch 1/10
19/19 [==============================] - 97s 5s/step - loss: 3.6755 - accuracy: 0.2533 - val_loss: 3.3230 - val_accuracy: 0.2866
Epoch 2/10
19/19 [==============================] - 93s 5s/step - loss: 0.8783 - accuracy: 0.7800 - val_loss: 3.3977 - val_accuracy: 0.3185
Epoch 3/10
19/19 [==============================] - 91s 5s/step - loss: 0.2841 - accuracy: 0.9267 - val_loss: 3.2138 - val_accuracy: 0.3312
Epoch 4/10
19/19 [==============================] - 91s 5s/step - loss: 0.2410 - accuracy: 0.9333 - val_loss: 2.8432 - val_accuracy: 0.4331
Epoch 5/10
19/19 [==============================] - 121s 7s/step - loss: 0.1539 - accuracy: 0.9467 - val_loss: 3.0529 - val_accuracy: 0.3694
Epoch 6/10
19/19 [==============================] - 121s 7s/step - loss: 0.1044 - accuracy: 0.9733 - val_loss: 2.9716 - val_accuracy: 0.3758
Epoch 7/10
19/19 [==============================] - 90s 5s/step - loss: 0.1110 - accuracy: 0.9600 - val_loss: 3.1626 - val_accuracy: 0.4140
Epoch 8/10
19/19 [

##Testing


In [ ]:
train.class_indices

{'blasti': 0,
 'bonegl': 1,
 'brhkyt': 2,
 'cbrtsh': 3,
 'cmnmyn': 4,
 'gretit': 5,
 'hilpig': 6,
 'himbul': 7,
 'himgri': 8,
 'hsparo': 9,
 'indvul': 10,
 'jglowl': 11,
 'lbicrw': 12,
 'mgprob': 13,
 'rebimg': 14,
 'wcrsrt': 15}

In [ ]:
output = ['blasti',
 'bonegl',
 'brhkyt',
 'cbrtsh',
 'cmnmyn',
 'gretit',
 'hilpig',
 'himbul',
 'himgri',
 'hsparo',
 'indvul',
 'jglowl',
 'lbicrw',
 'mgprob',
 'rebimg',
 'wcrsrt']

In [ ]:
img = image.load_img("/content/abc.jpg",target_size=(224,224))
img = image.img_to_array(img)
img = np.expand_dims(img,axis=0)
pred1 =  np.argmax(model.predict(img))
print(output[pred1])

1/1 [==============================] - 0s 108ms/step
blasti


In [ ]:

img1 = image.load_img("/content/abcd.jpg",target_size=(224,224))
img1 = image.img_to_array(img1)
img1 = np.expand_dims(img1,axis=0)
pred1_1 =  np.argmax(model.predict(img1))
print(output[pred1_1])

1/1 [==============================] - 0s 20ms/step
rebimg


In [ ]:

img = image.load_img("/content/spa.jpg",target_size=(224,224))
img = image.img_to_array(img)
img = np.expand_dims(img,axis=0)
pred2 =  np.argmax(model2.predict(img))
print(output[pred2])

1/1 [==============================] - 1s 660ms/step
cmnmyn
